In [ ]:
!pip install -i https://test.pypi.org/simple/ anguilla --upgrade

In [ ]:
!pip install jmetalpy

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import anguilla

from IPython.display import display, clear_output

import anguilla.hypervolume as hv

from anguilla.dominance import NonDominatedSet2D, NonDominatedSetKD
from anguilla.fitness import benchmark

pd.set_option('display.float_format', '{:.5E}'.format)

print(anguilla.__version__)

0.0.8


In [11]:
FNS_2D = [
    'ZDT1',
    'ZDT2',
    'ZDT3',
    'ZDT4',
    'ZDT6',
    'IHR1',
    'IHR2',
    'IHR3',
    'IHR4',
    'IHR6',
    'ELLI1',
    'ELLI2',
    'CIGTAB1',
    'CIGTAB2'
]

FNS_3D = [
    'DTLZ1',
    'DTLZ2',
    'DTLZ3',
    'DTLZ4',
    'DTLZ5',
    'DTLZ6',
    'DTLZ7',
    'GELLI',
]

ALGOS = [
    '(100+1)-MO-CMA-ES-I',
    '(100+1)-MO-CMA-ES-P',
    '(100+100)-MO-CMA-ES-I',
    '(100+100)-MO-CMA-ES-P'
]

In [12]:
class SummaryGenerator:

    def __init__(self, basepath, dim, fns, algos, n_evaluations):
        self.basepath = pathlib.Path(basepath)
        self.dim = dim
        self.fns = [fn for fn in fns if len(tuple(self.basepath.glob(f'**/{fn}*.fitness.csv'))) > 0]
        self.algos = algos
        self.n_evaluations = n_evaluations
        self._summary = None

    def _get_fn_paths(self, fn):
        paths = self.basepath.glob(f'**/{fn}*_{self.n_evaluations}.fitness.csv')
        return paths
    
    def _get_fn_algo_paths(self, fn, algo):
        paths = self.basepath.glob(f'**/{fn}_{algo}_*_{self.n_evaluations}.fitness.csv')
        return paths

    def _compute_reference_point(self, fn):
        paths = self._get_fn_paths(fn)
        point_set = NonDominatedSet2D() if self.dim == 2 else NonDominatedSetKD()
        for path in paths:
            points = np.genfromtxt(path, delimiter=',')
            point_set.insert(points)
        reference = point_set.upper_bound + 1.0
        return reference
    
    def _compute_median_indicator(self, fn, algo, reference):
        paths = self._get_fn_algo_paths(fn, algo)
        indicators = []
        for path in paths:
            points = np.genfromtxt(path, delimiter=',')
            indicator = hv.calculate(points, reference, ignore_dominated=True)
            #indicator = hv.shark_calculate(points, reference)
            indicators.append(indicator)
        return np.median(np.array(indicators))        

    def _compute_summary(self):
        rows = []
        for fn in self.fns:
            display(f'Computing reference point for: {fn}')
            reference = self._compute_reference_point(fn)
            display(f'Result: {reference}')
            row = []
            for algo in self.algos:
                display(f'Computing median HV indicator for: {algo}')
                median_indicator = self._compute_median_indicator(fn, algo, reference)
                display(f'Result: {median_indicator}')
                row.append(median_indicator)
            rows.append(row)
            clear_output()
        df = pd.DataFrame(rows, columns=self.algos, index=self.fns)
        return df
    
    
    def summary(self):
        if self._summary is None:
            self._summary = self._compute_summary()
        return self._summary
    
    def to_csv(self, name=None):
        if name is None:
            name = f'results_{self.n_evaluations}.csv'
        summary = self.summary()
        summary.to_csv(name)

In [13]:
s1 = SummaryGenerator('data/anguilla', 2, FNS_2D, ALGOS, 50000)
s1.summary()
s1.to_csv()

In [23]:
from jmetal.lab.statistical_test.functions import friedman_ph_test

result = friedman_ph_test(s1.summary())


In [24]:
result[0]

,(100+1)-MO-CMA-ES-I,(100+1)-MO-CMA-ES-P,(100+100)-MO-CMA-ES-I,(100+100)-MO-CMA-ES-P
(100+1)-MO-CMA-ES-I,0.00000E+00,1.46385E-01,2.48855E+00,1.46385E+00
(100+1)-MO-CMA-ES-P,1.46385E-01,0.00000E+00,2.63493E+00,1.61024E+00
(100+100)-MO-CMA-ES-I,2.48855E+00,2.63493E+00,0.00000E+00,1.02470E+00
(100+100)-MO-CMA-ES-P,1.46385E+00,1.61024E+00,1.02470E+00,0.00000E+00


In [25]:
result[1]

,(100+1)-MO-CMA-ES-I,(100+1)-MO-CMA-ES-P,(100+100)-MO-CMA-ES-I,(100+100)-MO-CMA-ES-P
(100+1)-MO-CMA-ES-I,1.00000E+00,8.83617E-01,1.28267E-02,1.43235E-01
(100+1)-MO-CMA-ES-P,8.83617E-01,1.00000E+00,8.41546E-03,1.07347E-01
(100+100)-MO-CMA-ES-I,1.28267E-02,8.41546E-03,1.00000E+00,3.05507E-01
(100+100)-MO-CMA-ES-P,1.43235E-01,1.07347E-01,3.05507E-01,1.00000E+00


In [ ]:
s2 = SummaryGenerator('data/shark', 2, FNS_2D, ALGOS, 50000)
s2.summary()

'Computing reference point for: CIGTAB2'

In [34]:
s3 = SummaryGenerator('data/anguilla', 3, FNS_3D, ALGOS, 50000)
s3.summary()

,(100+1)-MO-CMA-ES-I,(100+1)-MO-CMA-ES-P,(100+100)-MO-CMA-ES-I,(100+100)-MO-CMA-ES-P
DTLZ1,1.78711E+03,1.78711E+03,1.78711E+03,1.78711E+03
DTLZ2,6.53347E+00,6.56819E+00,6.77242E+00,6.54179E+00
DTLZ3,1.18968E+04,1.18968E+04,1.18968E+04,3.49711E+03
DTLZ4,6.66334E+00,6.66345E+00,6.66228E+00,6.61600E+00
DTLZ5,4.25907E+00,4.25901E+00,4.25777E+00,4.25790E+00
DTLZ6,4.25924E+00,4.25924E+00,4.25904E+00,4.25904E+00
DTLZ7,1.13348E+01,1.13347E+01,1.13331E+01,1.13315E+01


In [35]:
s4 = SummaryGenerator('data/shark', 3, FNS_3D, ALGOS, 50000)
s4.summary()

,(100+1)-MO-CMA-ES-I,(100+1)-MO-CMA-ES-P,(100+100)-MO-CMA-ES-I,(100+100)-MO-CMA-ES-P
DTLZ1,7.45838E+05,1.32319E+06,3.92158E+05,2.32029E+05
DTLZ2,5.58603E+00,6.70469E+00,9.27261E+00,9.33136E+00
DTLZ3,1.08077E+07,1.08077E+07,1.59237E+06,1.08077E+07
DTLZ4,7.48832E+00,7.05531E+00,1.04348E+01,1.05572E+01
DTLZ5,5.87198E+00,6.83431E+00,8.39971E+00,7.90444E+00
DTLZ6,2.35338E+00,5.82844E+00,4.25684E+00,4.20238E+00
DTLZ7,4.37958E+00,3.21968E+00,1.13424E+01,1.14202E+01
GELLI,NaN,NaN,NaN,NaN
